In [105]:
from pygame.locals import *
import pygame
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
import math


def window(material, geometry):
    pygame.init()
    pygame.display.set_caption('PyGame OpenGL')
    pygame.display.set_icon(pygame.image.load('icon.png'))
    screen = (800, 600)
    pygame.display.set_mode(screen, DOUBLEBUF|OPENGL)

    glEnable(GL_DEPTH_TEST)
    glEnable(GL_LIGHTING)
    glEnable(GL_LIGHT0)
    glEnable(GL_COLOR_MATERIAL)

    glMatrixMode(GL_PROJECTION)
    gluPerspective(45, (screen[0]/screen[1]), 0.1, 50.0)

    glMatrixMode(GL_MODELVIEW)  
    glTranslate(0.0,-1.5,0)
    # glTranslate(0.0,0,4)

    rot_x, rot_y, zoom = 30, 45, -10
    glClearColor(0, 0.4, 0.7, 1)

    clock = pygame.time.Clock()
    busy = True
    while busy:
        try:
            mouse_buttons = pygame.mouse.get_pressed()
            button_down = mouse_buttons[0] == 1
        
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    busy = False
                elif event.type == pygame.MOUSEMOTION:
                    if button_down:
                        rot_x = (rot_x + event.rel[1]) % 360
                        if rot_x > 90 and rot_x < 270:
                            rot_y = (rot_y - event.rel[0]) % 360
                        else:
                            rot_y = (rot_y + event.rel[0]) % 360
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    if event.button == 4:
                        zoom += 0.2
                    if event.button == 5:
                        zoom -= 0.2
                    
            glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
            glPushMatrix()
            glLight(GL_LIGHT0, GL_POSITION,  (0, 1, 0, 1))
            glLightfv(GL_LIGHT0, GL_AMBIENT,  (1, 1, 1, 0))
            glColorMaterial(GL_FRONT_AND_BACK, GL_AMBIENT_AND_DIFFUSE)
            glTranslatef(0.0,0.0, zoom)
            glRotatef(rot_x, 1, 0, 0)  
            glRotatef(rot_y, 0, 1, 0)  
            material(geometry)
            glPopMatrix()
            
            pygame.display.flip()
            clock.tick(100)
        except Exception as e:
            pygame.quit()
            raise e
            
    pygame.quit()

def basic_material(geometry):
    verticies, edegs, faces = geometry
    glBegin(GL_LINES)
    glColor3fv((1,1,1))
    for edeg in edegs:
        for vertex in edeg:
            glVertex3fv([v*1.004 for v in verticies[vertex]])
    glEnd()
    glBegin(GL_TRIANGLES)
    glColor3fv((0.4,0.4,0.4))    
    for face in faces:
        for vertex in face:
            glVertex3fv(verticies[vertex])
    glEnd()

def cube(shape):

    def rotate2(subject, pivot, angle):
        subject_x, subject_y = subject
        pivot_x, pivot_y = pivot
        a = angle * math.pi / 180
        dx = subject_x - pivot_x
        dy = subject_y - pivot_y
        r = math.sqrt(dx * dx + dy * dy)
        theta = math.atan2(dx, pivot_y - subject_y) - math.pi / 2
        return [
            pivot_x + r * math.cos(theta + a),
            pivot_y + r * math.sin(theta + a),
        ]
    
    def rotate(suject, pivot, angle):
        angle_x, angle_y, angle_z = angle
        pivot_x, pivot_y, pivot_z = pivot
        subject_x, subject_y, subject_z = suject

        subject_x, subject_z = rotate2([subject_x, subject_z], [pivot_x, pivot_z], angle_x)
        subject_y, subject_z = rotate2([subject_y, subject_z], [pivot_y, pivot_z], angle_y)
        subject_x, subject_y = rotate2([subject_x, subject_y], [pivot_x, pivot_y], angle_z)

        return [subject_x, subject_y, subject_z]
    def cauculate_center(points):
        center = [0,0,0]
        for i in points:
                center = [center[0] + i[0], center[1] + i[1], center[2] + i[2]]
        center = [center[0] / len(points), center[1] / len(points), center[2] / len(points)]
        return center
    verticies = []
    edegs = []
    faces = []
    index = 0
    previus_index = 0
    previus_plane = None
    z = -0.5
    previus_center = cauculate_center([[vertex[0], 0, vertex[1]] for vertex in shape[0][0]])
    rotation = [0, 0, 0]
    for section_id in range(len(shape)):
        plane, z_relative, scale, rotation_relative = shape[section_id]

        z += z_relative
        rotation = [it_rotation * 2 + itrotation_relative for it_rotation, itrotation_relative in zip(rotation, rotation_relative)]
        
        cur_verticies = [rotate([v[0], z, v[1]], previus_center, rotation) for v in plane]
        verticies += cur_verticies

        center = cauculate_center(cur_verticies)


        plane = [[j * scale for j in i] for i in plane]


        edegs += [[index+v, index+((v-1) % len(plane))] for v in range(len(plane))]



        if(index > 0):

            length = len(plane)
            previus_length = len(previus_plane)
            
            if(length > previus_length):
                bigger_length = length
                smaller_length = previus_length
            else:
                bigger_length = previus_length
                smaller_length = length

           
            if(length == previus_length):
                for j in range(bigger_length):
                    a = index + j
                    b = previus_index + j
                    pa = index + (j-1) % length
                    pb = previus_index + (j-1) % previus_length

                    edegs += [[a, b]]
                    edegs += [[pa, b]]

                    faces += [[a, b, pa]]
                    faces += [[pa, pb, b]]

            elif(length > previus_length):
                for j in range(bigger_length):
                    norm_b = j / length * previus_length

                    a  = index + math.floor(j)
                    b  = math.floor(previus_index + norm_b)
                    c = previus_index + math.ceil(norm_b) % smaller_length
                    pa = math.floor(index + ((j-1) % length))
                    
                    edegs += [[a, b]]
                    edegs += [[a, c]]

                    faces += [[a, b, c]]
                    faces += [[a, b, pa]]

            else:
                for j in range(bigger_length):
                    norm_a = j / previus_length * length

                    a  = index + math.floor(norm_a)
                    b  = math.floor(previus_index + j)
                    c = index + math.ceil(norm_a) % smaller_length
                    pb = math.floor(previus_index + ((j-1) % previus_length))

                    edegs += [[a, b]]
                    edegs += [[c, b]]

                    faces += [[a, b, c]]
                    faces += [[a, b, pb]]
        
        previus_index = index
        previus_plane = plane
        previus_center = center


        if section_id == 0 or section_id == len(shape)-1:

            verticies += [center]

            center_index = len(verticies)-1
            
            for i in range(len(plane)):
                edegs += [[center_index, index+i]]
                faces += [[center_index, index+i, index+((i-1) % len(plane))]]
            index += 1
        index += len(plane)
    # verticies += [[0,0,0] for i in range(10)]
    return verticies, edegs, faces


plane1 = [
    [-1, -1],
    [-1, 1],
    [1, 1],
    [1, -1],
]

plane2 = [
    [-1, -1],
    [-1, 1],
    [0, 2],
    [1, 1],
    [1, -1],
    [0, -2],
]

no_rotation = [0, 0, 0]
rotation = [0, 0, 20]


shape = [
    [plane1, 0,1, no_rotation],
    [plane2, 1.5,0.5, no_rotation],
    [plane2, 1.5,0.5, no_rotation],
    [plane2, 1.5,0.5, rotation],
    [plane1, 1.5,1,rotation],
]
window(basic_material, cube(shape))

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 20]
[0, 0, 60]
